<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark/examples/06-write_partitioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Write
- .write
- .format (parquet, csv, json)
- options
- spark.sql.sources.partitionOverwriteMode dynamic

# Write Mode
- overwrite - The overwrite mode is used to overwrite the existing file, alternatively, you can use SaveMode.Overwrite
- append - To add the data to the existing file, alternatively, you can use SaveMode.Append
- ignore - Ignores write operation when the file already exists, alternatively, you can use SaveMode.Ignore.
- errorifexists or error - This is a default option when the file already exists, it returns an error, alternatively, you can use SaveMode.ErrorIfExists.

# Partitioning
Process to organize the data into multiple chunks based on some criteria.
Partitions are organized in sub-folders.
Partitioning improves performance in Spark.

# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').getOrCreate()

# Preparing data

In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.6 MB/s eta 0:00:00


In [21]:
from faker import Faker
from datetime import datetime

fake = Faker()

users = []
for _ in range(50):
    user = {
        'date': fake.date_time_between_dates(datetime(2024, 5, 1), datetime(2024, 5, 5)),
        'name': fake.name(),
        'address': fake.address(),
        'email': fake.email(),
        'dob': fake.date_of_birth(),
        'phone': fake.phone_number()
    }
    users.append(user)

df = spark.createDataFrame(users)

df.show(10, False)


+-------------------------------------------------+--------------------------+----------+--------------------------+-------------------+--------------------+
|address                                          |date                      |dob       |email                     |name               |phone               |
+-------------------------------------------------+--------------------------+----------+--------------------------+-------------------+--------------------+
|USCGC Garza\nFPO AE 93586                        |2024-05-01 01:48:40.572505|1985-03-17|tinaperry@example.net     |Connie Miller      |966-288-5241        |
|327 Harris Mall\nRichstad, TX 25133              |2024-05-02 11:43:30.828588|1939-06-04|timothy69@example.net     |Bryan Cook         |(464)497-5312x67440 |
|Unit 0282 Box 3495\nDPO AA 47314                 |2024-05-04 00:33:05.895412|1992-07-30|qward@example.net         |Crystal Smith      |(430)385-0871x4879  |
|6337 Jennifer Center\nTravischester, OH 54107    |2

# Writing as PARQUET



In [113]:
# Writing as PARQUET with no partitions

path = "/content/write_partitioning/parquet_no_partitions"

df.write.mode("overwrite").format("parquet").save(path)

!ls /content/write_partitioning/parquet_no_partitions

spark.read.format("parquet").load(path).count()

part-00000-6dd38cf4-8853-4102-8a0b-f85c947e5dd5-c000.snappy.parquet  _SUCCESS


50

In [78]:
# Writing as PARQUET with partitions
from pyspark.sql.functions import *

path = "/content/write_partitioning/parquet_with_partitions"

# Creating partition column
df = df.withColumn("date_part", date_format(col("date"), "yyyyMMdd"))

spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic") # enable dynamic partition overwrite - only overwrites partitions that are coming in the dataframe

(df#.where("date_part = '20240503'")
 .write
 .mode("overwrite")                                               # overwrites the entire path with the new data
 .partitionBy("date_part")                                        # partition the data by column - creates sub-folders for each partition
 .format("parquet")                                               # format of output
 .save(path))                                                     # path

!ls /content/write_partitioning/parquet_with_partitions

spark.read.format("parquet").load(path).count()

'date_part=20240501'  'date_part=20240502'  'date_part=20240503'  'date_part=20240504'	 _SUCCESS


50

In [46]:
# Checking single partition
spark.read.parquet("/content/write_partitioning/parquet_with_partitions/date_part=20240502").show()

+--------------------+--------------------+----------+--------------------+------------------+--------------------+
|             address|                date|       dob|               email|              name|               phone|
+--------------------+--------------------+----------+--------------------+------------------+--------------------+
|327 Harris Mall\n...|2024-05-02 11:43:...|1939-06-04|timothy69@example...|        Bryan Cook| (464)497-5312x67440|
|699 Troy Prairie ...|2024-05-02 11:30:...|2018-12-31| megan98@example.net|       Denise Sims|       (704)828-2189|
|58674 Moore Camp ...|2024-05-02 20:05:...|1990-07-13|hughesjessica@exa...|       Susan Avila|          9717264698|
|58801 Bradley Tun...|2024-05-02 07:41:...|1918-12-14|  krocha@example.net|Christopher Newton|001-608-643-1024x...|
|078 Carroll Cente...|2024-05-02 12:27:...|1969-11-01|kathrynguerrero@e...|  Mrs. Karla Doyle|        369.656.9791|
|56066 Brandon Man...|2024-05-02 12:00:...|2010-02-07| sgarcia@example.o

# Writing as CSV

https://spark.apache.org/docs/3.5.1/sql-data-sources-csv.html

In [77]:
df.count()

50

In [88]:
path = "/content/write_partitioning/csv_no_partitioning/"

# write as csv
(df
  .write
  .format("csv")
  .mode("overwrite")
  .option("delimiter", "|")
  .option("header", True)
  .save(path))

# listing files in the folder
!ls /content/write_partitioning/csv_no_partitioning/

# read as csv
(spark
  .read
  .options(sep="|", multiLine=True, header=True)
  .csv(path)
  .count())

part-00000-b9f8e227-afcd-4d04-a350-c01327241872-c000.csv  _SUCCESS


50

# Writing as JSON

https://spark.apache.org/docs/3.5.1/sql-data-sources-json.html

In [91]:
path = "/content/write_partitioning/json_no_partitioning/"

# write as json
(df
.write
.mode("overwrite")
.format("json")
.save("path"))

# listing files in the folder
!ls /content/write_partitioning/json_no_partitioning/

# read as json
(spark
  .read
  .json(path)
  .count())

part-00000-4278fb78-1cc4-4622-9bb4-6cdadc50a8f2-c000.json  _SUCCESS


50

In [95]:
# reading json as text
spark.read.text(path).show(10, False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                 |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"address":"USCGC Garza\nFPO AE 93586","date":"2024-05-01T01:48:40.572Z","dob":"1985-03-17","email":"tinaperry@example.net","name":"Connie Miller","phone":"966-288-5241","date_part":"20240501"}                                     |
|{"address":"327 Harris Mall\nRichstad, TX 25133","date":"2024-05-02

In [96]:
# reading json as text
spark.read.json(path).show(10, False)

+-------------------------------------------------+------------------------+---------+----------+--------------------------+-------------------+--------------------+
|address                                          |date                    |date_part|dob       |email                     |name               |phone               |
+-------------------------------------------------+------------------------+---------+----------+--------------------------+-------------------+--------------------+
|USCGC Garza\nFPO AE 93586                        |2024-05-01T01:48:40.572Z|20240501 |1985-03-17|tinaperry@example.net     |Connie Miller      |966-288-5241        |
|327 Harris Mall\nRichstad, TX 25133              |2024-05-02T11:43:30.828Z|20240502 |1939-06-04|timothy69@example.net     |Bryan Cook         |(464)497-5312x67440 |
|Unit 0282 Box 3495\nDPO AA 47314                 |2024-05-04T00:33:05.895Z|20240504 |1992-07-30|qward@example.net         |Crystal Smith      |(430)385-0871x4879  |
|633

In [111]:
# partition json data + saveAsTable

# Creating partition column
df = df.withColumn("date_part", date_format(col("date"), "yyyyMMdd"))

# write as json
(df.write
  .partitionBy("date_part")
  .mode("overwrite")
  .format("json")
  .saveAsTable("tbl_json_part"))

# read as json
spark.table("tbl_json_part").count()

# read as json
spark.sql("show partitions tbl_json_part").show()

+------------------+
|         partition|
+------------------+
|date_part=20240501|
|date_part=20240502|
|date_part=20240503|
|date_part=20240504|
+------------------+



# Append Mode

In [116]:
# Writing as PARQUET with APPEND

path = "/content/write_partitioning/parquet_append"

df.write.mode("append").format("parquet").save(path)

!ls /content/write_partitioning/parquet_append

spark.read.format("parquet").load(path).count()

part-00000-19403de2-076f-46ee-849e-cb4d71597ef7-c000.snappy.parquet
part-00000-3b17956e-ed3d-4cbe-bebf-6602d8e45c1d-c000.snappy.parquet
part-00000-dc9d4535-21b1-452d-97eb-cc6cc34a5870-c000.snappy.parquet
_SUCCESS


150